In [3]:
from datetime import datetime
import datetime as dt
import numpy as np
import pandas as pd
from scipy import interpolate
import scipy.stats as stats
import py_vollib.ref_python.black_scholes_merton.implied_volatility as implied_volatility
import py_vollib.black.implied_volatility as implied_volatility2
from decimal import *
getcontext().prec = 6

In [4]:
shibor_rate = pd.read_csv('data/shibor.csv', index_col=0, encoding='GBK')
tradeday = pd.read_csv('data/tradetime.csv', encoding='GBK')
pd.options.mode.chained_assignment = None

In [46]:
def periodsSplineRiskFreeInterestRate(options, date):
    """
    params: options: 计算VIX的当天的options数据用来获取expDate
            date: 计算哪天的VIX
    return：shibor：该date到每个到期日exoDate的risk free rate

    """
    date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    date = datetime(date.year,date.month,date.day)
    exp_dates = np.sort(options.maturity_date.unique())
    periods = {}
    for epd in exp_dates:
        epd = pd.to_datetime(epd)
        periods[epd] = ((epd - date).days+1) * 1.0 / 365.0
    shibor_date = datetime.strptime(shibor_rate.index[0], "%Y-%m-%d")
    if date >= shibor_date:
        date_str = shibor_rate.index[0]
        shibor_values = shibor_rate.loc[0].values
        # shibor_values = np.asarray(list(map(float,shibor_values)))
    else:
        date_str = date.strftime("%Y-%m-%d")
        shibor_values = shibor_rate.loc[date_str].values
        # shibor_values = np.asarray(list(map(float,shibor_values)))
    shibor = {}
    period = np.asarray([1.0, 7.0, 14.0, 30.0, 90.0, 180.0, 270.0, 360.0]) / 360.0
    min_period = min(period)
    max_period = max(period)
    sh_func = interpolate.interp1d(period, shibor_values,kind = 'cubic')
    for p in periods.keys():
        tmp = periods[p]
        if periods[p] > max_period:
            tmp = max_period * 0.99999
        elif periods[p] < min_period:
            tmp = min_period * 1.00001
        # 此处使用SHIBOR来插值
        sh = sh_func(tmp)
        shibor[p] = sh / 100.0
    return shibor


def getHistDayOptions(vixDate, options_data):
    options_data = options_data.loc[vixDate,:]
    return options_data


def getNearNextOptExpDate(options, vixDate):
    # 找到options中的当月和次月期权到期日；
    # 用这两个期权隐含的未来波动率来插值计算未来30隐含波动率，是为市场恐慌指数VIX；
    # 如果options中的最近到期期权离到期日仅剩1天以内，则抛弃这一期权，改选择次月期权和次月期权之后第一个到期的期权来计算。
    # 返回的near和next就是用来计算VIX的两个期权的到期日
    """
    params: options: 该date为交易日的所有期权合约的基本信息和价格信息
            vixDate: VIX的计算日期
    return: near: 当月合约到期日（ps：大于1天到期）
            next：次月合约到期日
    """
    vixDate = datetime.strptime(vixDate, '%Y-%m-%d %H:%M:%S')
    optionsExpDate = list(pd.Series(options.maturity_date.values.ravel()).unique())
    optionsExpDate = [datetime.strptime(i, '%Y-%m-%d') for i in optionsExpDate]
    near = min(optionsExpDate)
    optionsExpDate.remove(near)
    if (near - vixDate).days < 7:
        near = min(optionsExpDate)
        optionsExpDate.remove(near)
    nt = min(optionsExpDate)
    #set it to 8:30 AM
    near_final = datetime(near.year,near.month,near.day,8,30,0,0)
    nt_final = datetime(nt.year,nt.month,nt.day,8,30,0,0)
    return near_final, nt_final


def getStrikeMinCallMinusPutClosePrice(options):
    # options 中包括计算某日VIX的call和put两种期权，
    # 对每个行权价，计算相应的call和put的价格差的绝对值，
    # 返回这一价格差的绝对值最小的那个行权价，
    # 并返回该行权价对应的call和put期权价格的差
    """
    params:options: 该date为交易日的所有期权合约的基本信息和价格信息
    return: strike: 看涨合约价格-看跌合约价格 的差值的绝对值最小的行权价
            priceDiff: 以及这个差值，这个是用来确定中间行权价的第一步
    """
    call = options[options.option_type == u"C"].set_index(u"strike_price").sort_index()
    put = options[options.option_type == u"P"].set_index(u"strike_price").sort_index()
    callMinusPut = call.close - put.close
    strike = abs(callMinusPut).idxmin()
    priceDiff = callMinusPut[strike]
    return strike, priceDiff


def calSigmaSquare(options, FF, R, T):
    # 计算某个到期日期权对于VIX的贡献sigma；
    # 输入为期权数据options，FF为forward index price，
    # R为无风险利率， T为期权剩余到期时间
    """
    params: options:该date为交易日的所有期权合约的基本信息和价格信息
            FF: 根据上一步计算得来的strike，然后再计算得到的forward index price， 根据它对所需要的看涨看跌合约进行划分。
                取小于FF的第一个行权价为中间行权价K0， 然后选取大于等于K0的所有看涨合约， 选取小于等于K0的所有看跌合约。
                对行权价为K0的看涨看跌合约，删除看涨合约，不过看跌合约的价格为两者的均值。
            R： 这部分期权合约到期日对应的无风险利率 shibor
            T： 还有多久到期（年化）
    return：Sigma：得到的结果是传入该到期日数据的Sigma
    """
    callAll = options[options.option_type == u"C"].set_index(u"strike_price").sort_index()
    putAll = options[options.option_type == u"P"].set_index(u"strike_price").sort_index()
    callAll['deltak'] = 0.05
    putAll['deltak'] = 0.05
    k0 = callAll.loc[:FF].index.max()
    call = callAll[callAll.index >= k0]
    put = putAll[putAll.index <= k0]
    calldict = call['close'].to_dict()
    calldict_clean = check_preprocessing(calldict)
    putdict = put['close'].to_dict()
    putdict_clean = check_preprocessing(putdict)
    k0_idx = k0
    callsds = implied_vol('c', call['close'].loc[k0_idx], FF, k0, R, T, R)
    putsds = implied_vol('p', put['close'].loc[k0_idx], FF, k0, R, T, R)
#     sds = ((callsds + putsds) / 2) * np.sqrt(T) * FF
    if putdict == {}:
        k0_idx = call.index[0]
        call, delta = interpolate_volatility(call,5,FF,R,T,R,'C',sds)
        call['deltak'] = delta
        callComponent = call.close * call.deltak / call.index / call.index
        sigma = (sum(callComponent)) * np.exp(T * R) * 2 / T
        sigma = sigma - (FF / k0_idx - 1) ** 2 / T
    elif calldict == {}:
        k0_idx = put.index[-1]
        put,delta = interpolate_volatility(put,5,FF,R,T,R,'P',sds)
        put['deltak'] = delta
        putComponent = put.close * put.deltak / put.index / put.index
        sigma = (sum(putComponent)) * np.exp(T * R) * 2 / T
        sigma = sigma - (FF / k0_idx - 1) ** 2 / T
    else:
        k0_idx = put.index[-1]
        put['close'].iloc[-1] = (putAll.loc[k0_idx].close + callAll.loc[k0_idx].close) / 2.0
        
        call, deltacall = interpolate_volatility(call,5,FF,R,T,R,'C',sds)
        put, deltaput = interpolate_volatility(put,5,FF,R,T,R,'P',sds)
        call['deltak'] = deltacall
        put['deltak'] = deltaput
        callComponent = call.close * call.deltak / call.index / call.index
        putComponent = put.close * put.deltak / put.index / put.index
        sigma = (sum(callComponent) + sum(putComponent)) * np.exp(T * R) * 2 / T
        sigma = sigma - (FF / k0_idx - 1) ** 2 / T
        return sigma, (callsds + putsds) / 2

def changeste(t):
    if t.month >= 10:
        str_t = t.strftime('%Y-%m-%d')
    else:
        str_t = t.strftime('%Y-%m-%d')
        # str_t = str_t[:5] + str_t[6:]
    return str_t


def calDayVIX(vixDate):
    # 利用CBOE的计算方法，计算历史某一日的未来30日期权波动率指数VIX
    """
    params：vixDate：计算VIX的日期  '%Y/%m/%d' 字符串格式
    return：VIX结果
    """

    # 拿取所需期权信息
    date = datetime.strptime(vixDate, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d')
    options_data = pd.read_csv('data/new_data/'+date+'.csv', index_col='datetime')
    options = getHistDayOptions(vixDate, options_data)
    near, nexts = getNearNextOptExpDate(options, vixDate)
    shibor = periodsSplineRiskFreeInterestRate(options, vixDate)
    R_near = shibor[datetime(near.year, near.month, near.day)]
    R_next = shibor[datetime(nexts.year, nexts.month, nexts.day)]
    str_near = changeste(near)
    str_nexts = changeste(nexts)
    optionsNearTerm = options[options.maturity_date == str_near]
    optionsNextTerm = options[options.maturity_date == str_nexts]
    # time to expiration
    vixDate = datetime.strptime(vixDate, '%Y-%m-%d %H:%M:%S')
    T_near = (near - vixDate).total_seconds() / (365.0*24.0*60.0*60.0)
    T_next = (nexts - vixDate).total_seconds() / (365.0*24.0*60.0*60.0)
    # the forward index prices
    nearMinStrike, nearPriceDiff = getStrikeMinCallMinusPutClosePrice(optionsNearTerm)
    nextMinStrike, nextPriceDiff = getStrikeMinCallMinusPutClosePrice(optionsNextTerm)
    near_F = nearMinStrike + np.exp(T_near * R_near) * nearPriceDiff
    next_F = nextMinStrike + np.exp(T_next * R_next) * nextPriceDiff
    # 计算不同到期日期权对于VIX的贡献
#     near_ntm_percentage, near_sigma,near_ntmsigma = calSigmaSquare(optionsNearTerm, near_F, R_near, T_near)
#     next_ntm_percentage,next_sigma,next_ntmsigma = calSigmaSquare(optionsNextTerm, next_F, R_next, T_next)
    near_sigma, near_BSIV = calSigmaSquare(optionsNearTerm, near_F, R_near, T_near)
    next_sigma, next_BSIV = calSigmaSquare(optionsNextTerm, next_F, R_next, T_next)

    # 利用两个不同到期日的期权对VIX的贡献sig1和sig2，、
    # 已经相应的期权剩余到期时间T1和T2；
    # 差值得到并返回VIX指数(%)
    w = (T_next - 30.0 / 365.0) / (T_next - T_near)
    vix = T_near * w * near_sigma + T_next * (1 - w) * next_sigma
    return 100 * np.sqrt(abs(vix) * 365.0 / 30.0),near_BSIV, next_BSIV

In [6]:
def check_preprocessing(inputdict):
    returndict = inputdict.copy()
    for s in list(returndict.keys()):
        if returndict[s] <= 0.0002:
            del returndict[s]
    return returndict
def implied_vol(option_type, option_price, s, k, r, T, q):
    # apply bisection method to get the implied volatility by solving the BSM function
    precision = 0.00001
    upper_vol = 500.0
    max_vol = 500.0
    min_vol = 0.0001
    lower_vol = 0.0001
    iteration = 0
    while True:
        iteration +=1
        mid_vol = (upper_vol + lower_vol)/2.0
        price = bsm_price(option_type, mid_vol, s, k, r, T, q)
        if option_type == 'c':
            lower_price = bsm_price(option_type, lower_vol, s, k, r, T, q)
            if (lower_price - option_price) * (price - option_price) > 0:
                lower_vol = mid_vol
            else:
                upper_vol = mid_vol
            if abs(price - option_price) < precision: break 
            if mid_vol > max_vol - 5 :
                mid_vol = 0.000001
                break
        elif option_type == 'p':
            upper_price = bsm_price(option_type, upper_vol, s, k, r, T, q)

            if (upper_price - option_price) * (price - option_price) > 0:
                upper_vol = mid_vol
            else:
                lower_vol = mid_vol
            if abs(price - option_price) < precision: break 
            if iteration > 50: break

    return mid_vol
def bsm_price(option_type, sigma, s, k, r, T, q):
    # calculate the bsm price of European call and put options
    sigma = float(sigma)
    d1 = (np.log(s / k) + (r - q + sigma ** 2 * 0.5) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'c':
        price = np.exp(-r*T) * (s * np.exp((r - q)*T) * stats.norm.cdf(d1) - k *  stats.norm.cdf(d2))
        return price
    elif option_type == 'p':
        price = np.exp(-r*T) * (k * stats.norm.cdf(-d2) - s * np.exp((r - q)*T) *  stats.norm.cdf(-d1))
        return price
    else:
        print('No such option type %s') %option_type
def insert_array(k,array):
    to_return = []
    for i in range(len(array)-1):
        to_return.append(array[i])
        step = array[i+1]-array[i]
        for j in range(k-1):
            to_return.append(array[i]+(j+1)*step/k)
    to_return.append(array[-1])
    return to_return
def insert_strike(inputdict,target):
    to_return = {}
    for p in target:
        st_func = interpolate.interp1d(list(inputdict.keys()), list(inputdict.values()))
        to_return[p] = float(st_func(p))
    return to_return
def interpolate_volatility(x,k,F, r, T, q, option_type,sds):
# interpolation
    try:
        index = insert_array(5,x.index)
        interpolation = insert_strike(x.close.to_dict(),index)
        delta = index[1]-index[0]
        to_return = pd.DataFrame(data=list(interpolation.values()),index=list(interpolation.keys()),columns = ['close'])
    except:
        to_return = pd.DataFrame(data=x.close, index = x.index)
        delta = 0.05/k
# extrapolation
    if option_type == 'C':
        idx = to_return.index[-1]
        last_vol = implied_vol('c', to_return.close[idx], F, idx, r, T, q)
        while idx - to_return.index[0] < 5*sds:
            to_return.loc[idx + delta] = bsm_price('c', last_vol, F, idx + delta , r, T, q)
            idx = idx + delta
        to_return = to_return.drop([to_return.index[0]])
    else:
        idx = to_return.index[0]
        first_vol = implied_vol('p', to_return.close[idx], F, idx, r, T, q)
        while to_return.index[-1] - idx < 5*sds and idx > 0.0001:
            to_return.loc[idx-delta] = bsm_price('p', first_vol, F, idx - delta , r, T, q)
            idx = idx - delta
    to_return = to_return.sort_index()
    return to_return, delta

In [ ]:
for day in tradeday['datetime']:
    c, near_BSIV, next_BSIV = calDayVIX(day)
    print(day, c, near_BSIV, next_BSIV)